# Spacy sentence transformers demo 3

Status: not working, something misaligned with nlp and tensors

In [1]:
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal

In [2]:
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using gpu!")
    # torch.set_default_tensor_type("torch.cuda.FloatTensor")

Using gpu!


**Note:** it seems en_core_web_sm does not have vectors (for size reduction), I had to use '_lg' to get the tensors.shape to work

In [10]:
s_model = 'en_core_web_lg'
nlp = spacy.load(s_model)

In [12]:
doc = nlp("Here is some text to encode.")
doc

Here is some text to encode.

In [14]:
doc.tensor.shape

(7, 96)

In [15]:
doc.has_vector
doc.vector

array([ 4.2845517e-01,  2.0716856e+00, -5.6854564e-01, -1.4164957e+00,
        2.7636120e+00,  1.2712100e+00,  8.3245289e-01,  2.5122435e+00,
       -3.7793028e+00, -7.3004007e-01,  9.0821857e+00,  1.6326271e+00,
       -4.9689140e+00,  2.8948987e+00,  7.5091714e-01,  1.6071113e+00,
        1.6758429e+00, -1.6004571e+00, -2.0532000e+00, -2.7825546e+00,
       -2.0497139e-01, -8.7526286e-01, -1.0981814e+00, -1.2076771e+00,
       -2.6530426e+00, -1.4273437e+00, -2.1059201e+00, -2.4124856e+00,
       -3.3352273e+00,  1.9563740e+00,  7.7312857e-01, -6.6265839e-01,
       -3.9945140e+00, -3.0432887e+00, -2.5132744e+00,  1.0765145e+00,
       -1.6529428e+00,  1.3278530e+00,  3.1464956e+00,  2.8807142e+00,
        1.1837599e+00,  1.7849790e+00, -1.1852857e+00, -4.3203428e-01,
       -3.0609043e+00,  3.2149942e+00,  4.8887787e+00, -2.3499000e+00,
       -3.6770146e+00,  1.8782552e+00, -2.8511712e-01, -5.6754291e-01,
        1.7457840e+00, -5.9228888e+00, -1.3169800e+00,  3.1983998e-01,
      

This fails, it is a different shape (perhaps copy/paste oversight on my part?)

In [16]:
assert doc.tensor.shape == (7, 768)  # Always has one row per token

AssertionError: 

In [17]:
doc._.trf_word_pieces_  # String values of the wordpieces
# doc._.trf_word_pieces  # Wordpiece IDs (note: *not* spaCy's hash values!)
# doc._.trf_alignment  # Alignment between spaCy tokens and wordpieces

AttributeError: [E046] Can't retrieve unregistered extension attribute 'trf_word_pieces_'. Did you forget to call the `set_extension` method?

In [ ]:
# The raw transformer output has one row per wordpiece.
assert len(doc._.trf_last_hidden_state) == len(doc._.trf_word_pieces)

In [ ]:
# To avoid losing information, we calculate the doc.tensor attribute such that
# the sum-pooled vectors match (apart from numeric error)
assert_almost_equal(doc.tensor.sum(axis=0), doc._.trf_last_hidden_state.sum(axis=0), decimal=5)
span = doc[2:4]

In [ ]:
# Access the tensor from Span elements (especially helpful for sentences)
assert numpy.array_equal(span.tensor, doc.tensor[2:4])
# .vector and .similarity use the transformer outputs
apple1 = nlp("Apple shares rose on the news.")
apple2 = nlp("Apple sold fewer iPhones this quarter.")
apple3 = nlp("Apple pie is delicious.")
print(apple1[0].similarity(apple2[0]))  # 0.73428553
print(apple1[0].similarity(apple3[0]))  # 0.43365782